# 1. 導入套件

In [1]:
# 主要套件
import torch

# 資料處理
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 建立機器模型
from torch import nn
from torch import optim

# 訓練過程視覺化
from torch.utils.tensorboard import SummaryWriter

# 2. 修正參數

In [2]:
# 擷取資料檔案
gesture_data_file = "dataset/train_data-x_y.csv"

In [3]:
# 取得 GPU 或是 CPU 的設備進行訓練
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
# 批次
batch_size = 64

In [5]:
# 優化函數
optim_fn = optim.SGD
# 損失函數
loss_fn = nn.CrossEntropyLoss()

In [6]:
# 學習率
learning_rate = 1e-5
# 世代
epochs = 60

In [7]:
# TensorBoard 的物件
writer = SummaryWriter()

# 3. 類別

In [8]:
# 姿態資料集
class GestureDataset(Dataset):
    def __init__(self, df:pd.DataFrame):
        self.dataframe = df
        pass
    
    def __getitem__(self, idx):
        # 取得資料特徵，轉換成 torch 格式，資料為 torch.float32 格式
        feature = torch.tensor(self.dataframe.iloc[idx, :-1]).to(torch.float32)
        # 取得結果
        result = self.dataframe.iloc[idx, -1]
        return feature, result
    
    def __len__(self):
        return len(self.dataframe)

In [9]:
# 姿態模型
class GestureModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.relu_linear_stack = nn.Sequential(
            nn.Linear(34, 68),
            nn.Dropout(p=0.1),
            nn.ReLU(),
            nn.Linear(68, 4)
        )
        pass
    
    def forward(self, x):
        logits = self.relu_linear_stack(x)
        return logits

# 4. 函數

In [10]:
# 訓練函數
def train(dataloader:DataLoader, model:nn.Module, loss_fn:torch.nn.modules.Module, optimizer:optim.Optimizer):
    size = len(dataloader.dataset)
    
    # 開始訓練
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 轉換成可讓模型訓練的格式
        X, y = X.to(device), y.to(device)
        
        # 計算預測誤差
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation 反向傳播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 輸出
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
# 測試函數
# 回傳成功率，測試 loss
def test(dataloader:DataLoader, model:nn.Module, loss_fn:torch.nn.modules.Module):
    
    # 計算結果參數
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 測試誤差、正確率
    test_loss, correct = 0, 0
    
    # 開始驗證
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            # 轉換成可讓模型訓練的格式
            X, y = X.to(device), y.to(device)

            # 計算預測誤差
            pred = model(X)
            loss = loss_fn(pred, y)
            
            # 計算誤差、正確率
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return correct*100, test_loss

In [12]:
# 輔助函式
from playsound import playsound
# 撥放音樂
def PlaySound():
    sound_pth = "E:\Media Cabinet\Musics\Musics\dio zawaruto.mp3"
    playsound(sound_pth)

# 5. 使用資料

In [13]:
# 讀取 csv 資料
gesture_df = pd.read_csv(gesture_data_file)

# 切分成訓練以及測試資料
train_df = gesture_df.iloc[::3]
test_df = gesture_df.iloc[1::2]

# 建立資料集
train_dataset = GestureDataset(train_df)
test_dataset = GestureDataset(test_df)

# 建立加載器
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 6. 建立模型

In [14]:
# 建立模型
model = GestureModel().to(device)
model

GestureModel(
  (relu_linear_stack): Sequential(
    (0): Linear(in_features=34, out_features=68, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=68, out_features=4, bias=True)
  )
)

# 7. 訓練模型

In [15]:
# 訓練模型
optimizer = optim_fn(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    
    # 使用 tensorboard 印出訓練正確率
    train_accuracy, train_loss = test(train_dataloader, model, loss_fn)
    writer.add_scalar('Accuracy/train', train_accuracy, t)
    writer.add_scalar('Loss/train', train_loss, t)
    
    # 使用 tensorboard 印出驗證正確率
    # test_accuracy, test_loss = test(test_dataloader, model, loss_fn)
    # writer.add_scalar('Accuracy/test', test_accuracy, t)
    # writer.add_scalar('Loss/test', test_loss, t)
    
print("Done!")
writer.flush() 結束繪圖
# PlaySound() # 完成訓練後撥放音樂

Epoch 1
-------------------------------
loss: 102.679001 [   64/ 1179]
Test Error: 
 Accuracy: 28.0%, Avg loss: 28.003799 

Epoch 2
-------------------------------
loss: 56.914711 [   64/ 1179]
Test Error: 
 Accuracy: 39.2%, Avg loss: 23.907718 

Epoch 3
-------------------------------
loss: 68.397827 [   64/ 1179]
Test Error: 
 Accuracy: 44.6%, Avg loss: 26.594154 

Epoch 4
-------------------------------
loss: 42.720608 [   64/ 1179]
Test Error: 
 Accuracy: 46.3%, Avg loss: 22.789234 

Epoch 5
-------------------------------
loss: 35.691982 [   64/ 1179]
Test Error: 
 Accuracy: 43.9%, Avg loss: 26.533219 

Epoch 6
-------------------------------
loss: 43.981949 [   64/ 1179]
Test Error: 
 Accuracy: 47.8%, Avg loss: 15.699013 

Epoch 7
-------------------------------
loss: 30.647161 [   64/ 1179]
Test Error: 
 Accuracy: 50.0%, Avg loss: 9.966164 

Epoch 8
-------------------------------
loss: 32.610931 [   64/ 1179]
Test Error: 
 Accuracy: 48.7%, Avg loss: 16.742963 

Epoch 9
--------